In [1]:
sc.stop()
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

conf = SparkConf().setAppName("202")\
                  .setMaster("spark://10.120.26.200:7077")\
                  .set("spark.executor.memory", '6g')\
                  .set('spark.cores.max', '4')

sc = SparkContext(conf=conf)
spark = SparkSession.builder.appName('sql coming~').getOrCreate()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
50,None,pyspark,idle,,,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
finances_df = spark.read.json("hdfs://10.120.26.200/user/spark/spark_sql_101/data/finances.json")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
etl_df = finances_df.na.drop("all", subset=["ID", "Account", "Amount", "Description", "Date"]) \
                        .na.fill("Unknown", "Description") \
                        .where((finances_df["Amount"] != 0) | (finances_df["Description"] == "Unknown")) \
                        .select("Account.Number", "Amount", "Date", "Description")
etl_df.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+------+---------+--------------------+
|     Number|Amount|     Date|         Description|
+-----------+------+---------+--------------------+
|123-ABC-789|  1.23| 1/1/2015|          Drug Store|
|456-DEF-456| 200.0| 1/3/2015|         Electronics|
|123-ABC-789|  2.36| 1/9/2015|       Grocery Store|
|456-DEF-456| 23.16|1/11/2015|             Unknown|
|123-ABC-789| 42.12|1/12/2015|                Park|
|456-DEF-456|  20.0|1/12/2015|         Electronics|
|333-XYZ-999| 52.13|1/17/2015|                 Gas|
|333-XYZ-999| 41.67|1/19/2015|Some Totally Fake...|
|333-XYZ-999| 56.37|1/21/2015|                 Gas|
|987-CBA-321| 63.84|1/23/2015|       Grocery Store|
|123-ABC-789|160.91|1/24/2015|         Electronics|
|456-DEF-456| 78.77|1/24/2015|       Grocery Store|
|333-XYZ-999| 86.24|1/29/2015|              Movies|
|456-DEF-456| 93.71|1/31/2015|       Grocery Store|
|987-CBA-321|  2.29|1/31/2015|          Drug Store|
|456-DEF-456|108.64|1/31/2015|                Park|
|456-DEF-456

In [4]:
corrupt_df = finances_df.where(finances_df["_corrupt_record"].isNotNull()).select("_corrupt_record", "ID")
corrupt_df.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+----+
|     _corrupt_record|  ID|
+--------------------+----+
|{"ID":3,"Account"...|null|
+--------------------+----+

In [5]:
account_df = finances_df.select(concat("Account.FirstName",lit(" "),"Account.LastName").alias("FullName"),
                                           finances_df["Account.Number"].alias("AccountNumber")) \
                            .na.drop("all") \
                            .distinct()
account_df.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+-------------+
|     FullName|AccountNumber|
+-------------+-------------+
|    Jay Smith|  123-ABC-789|
| Sally Fuller|  456-DEF-456|
|Justin Pihony|  987-CBA-321|
|  Brad Turner|  333-XYZ-999|
+-------------+-------------+

In [7]:

date = finances_df.select(to_date(unix_timestamp(finances_df["Date"], "MM/dd/yyyy").cast("timestamp")).alias("Date"))
date.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+
|      Date|
+----------+
|2015-01-01|
|2015-01-03|
|      null|
|2015-01-04|
|2015-01-09|
|2015-01-11|
|2015-01-12|
|2015-01-12|
|2015-01-17|
|2015-01-19|
|2015-01-21|
|2015-01-23|
|2015-01-24|
|2015-01-24|
|2015-01-29|
|2015-01-31|
|2015-01-31|
|2015-01-31|
|2015-01-31|
|2015-02-10|
+----------+
only showing top 20 rows

In [8]:
account_details_df = finances_df.select(finances_df["Account.Number"].alias("AccountNumber"),
                                            finances_df["Amount"],
                                            finances_df["Description"],
                                            to_date(unix_timestamp(finances_df["Date"], "MM/dd/yyyy").cast("timestamp")).alias("Date")) \
                                     .na.drop("all", subset=["AccountNumber", "Amount", "Description", "Date"]) \
                                     .groupBy("AccountNumber") \
                                     .agg(avg("Amount").alias("AverageTransaction"), sum("Amount").alias("TotalTransactions"),
                                          count("Amount").alias("NumberOfTransactions"), max("Amount").alias("MaxTransaction"),
                                          min("Amount").alias("MinTransaction"), stddev("Amount").alias("StandardDeviationAmount"),
                                          collect_set("Description").alias("UniqueTransactionDescriptions"))

account_details_df.show(truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+------------------+------------------+--------------------+--------------+--------------+-----------------------+-----------------------------+
|AccountNumber|AverageTransaction| TotalTransactions|NumberOfTransactions|MaxTransaction|MinTransaction|StandardDeviationAmount|UniqueTransactionDescriptions|
+-------------+------------------+------------------+--------------------+--------------+--------------+-----------------------+-----------------------------+
|  456-DEF-456|104.75142857142855|1466.5199999999998|                  14|         281.0|          6.78|      86.07276714475982|         [Electronics, Gro...|
|  333-XYZ-999|103.92545454545456|           1143.18|                  11|         241.8|         41.67|      77.92123104310356|         [Electronics, Gro...|
|  987-CBA-321| 87.19099999999999| 871.9099999999999|                  10|        267.93|           0.0|      86.26235350500369|         [Electronics, Gro...|
|  123-ABC-789| 362.9785714285714|            

In [ ]:
account_details_df.show(truncate=False)